In [1]:
from langchain_community.llms import Ollama

In [2]:
#initialize the model
llm = Ollama(model='tinyllama')

/var/folders/dz/qr8lbzs568b3m37bzckypts80000gn/T/ipykernel_98019/1923718779.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  llm = Ollama(model='tinyllama')


In [3]:
# generate answers to questions
question = "when did india get independence"
response = llm.invoke(question)
print(response)

India achieved independence on August 15, 1947, which is celebrated as Independence Day every year in India. The country gained its freedom from British colonial rule after more than 300 years of British rule. On this day, people across the country celebrate by lighting candles or lamp-bearers, parading through the streets with flags and other symbols of national pride, singing patriotic songs, and engaging in various cultural activities.


IMPLEMENTING RAG

In [4]:

import openai 
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS


In [6]:
pdf_reader = PyPDFLoader('paper.pdf')
documents = pdf_reader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
)
chunks= text_splitter.split_documents(documents)

In [7]:
# create embeddings using pre-HF model
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
db = FAISS.from_documents(documents=chunks, embedding=embeddings)


/var/folders/dz/qr8lbzs568b3m37bzckypts80000gn/T/ipykernel_98019/3867230902.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


In [9]:
print(len(db.docstore._dict))

83


In [11]:
# Access the first document chunk
first_doc = list(db.docstore._dict.values())[0]
print(first_doc.page_content[:500])  # Print first 500 characters

Regular Manuscript
Bimodal Multi-Scale Attention Autoencoder (Bi-MSAAE) for 
Facial Image and Video Denoising
Submission ID a6b7c002-2e49-4f21-81af-4bd3a3ab2c0b
Submission Version Initial Submission
PDF Generation 28 Oct 2025 09:44:39 EST by Atypon ReX
Authors
Mr. RAJVEER JADAV
Affiliations
 Dept. of Electrical and Computer Engineering, 
University of Waterloo, Waterloo, ON, CA
Dr. MARZIA ZAMAN
Affiliations
 Research & Development Department, EdgesAI 
Technology, Ottawa, ON, CA
Dr. DARSHANA UP


In [21]:
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama


In [29]:
def retrieve_answer(query, top_k=3):
    # Get top-k most similar document chunks
    results = db.similarity_search(query, k=top_k)
    
    # Combine their text into a prompt
    context = "\n\n".join([doc.page_content for doc in results])
    prompt = f"Use the following context to answer the question:\n{context}\n\nQuestion: {query}\nAnswer:"

    # Use .generate() with a list of prompts
    response = llm.generate([prompt])
    
    # Extract text from the first result
    return response.generations[0][0].text

# Example usage
question = "Who is Rajveer Jadav"
answer = retrieve_answer(question)
print("Answer:", answer)

Answer: The context provided in the given text includes an overview of Manzanoeet al.: Preparation of Papers for IEEE ACCESS, which contains full source code, model weights, experimental results, and supporting documentation for replication. The author, Rajveeer Jaadaver Jadav, is a B.Tech degree holder in Information Technology from Sri Siwasubra-maniyya Nadar College of Engineering, Chennai, India, and an M.Eng. Degree in Electric & Computer Engineering from the University of Waterloo, Canada. He has practical experience in MLOps, including CI/CD pipelines, AI model deployment, software testing, and cloud-based application deployment. His research interests include deep learning, machine learning operations (MLOp), large language models (LLMs), retrieval-augmented generation (RAG), model optimization, and secure AI systems.
